In [16]:
import pandas as pd
dataset = pd.read_csv("D:/桌面/Market_Basket_Optimisation.csv")
print(dataset.shape)
dataset.head()

(7500, 20)


,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# 转换Numpy数组类型
import numpy as np
transaction = []
for i in range(0, dataset.shape[0]):
    for j in range(0, dataset.shape[1]):
        transaction.append(dataset.values[i,j])
transaction = np.array(transaction)
print(transaction)

['burgers' 'meatballs' 'eggs' ... 'nan' 'nan' 'nan']


In [18]:
import pandas as pd
df = pd.DataFrame(transaction, columns=["items"]) 
df["incident_count"] = 1 
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)
# 对商品名称进行分组，计算每种商品的购买总次数，取前5序号
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table.head(5).style.background_gradient(cmap='Greens')

,items,incident_count
0,mineral water,1787
1,eggs,1348
2,spaghetti,1306
3,french fries,1282
4,chocolate,1230


In [19]:
# 将每笔交易转换为单独的列表，并将它们收集到Numpy数组中
import numpy as np
transaction = []
for i in range(dataset.shape[0]):
    transaction.append([str(dataset.values[i,j]) for j in range(dataset.shape[1])])
transaction = np.array(transaction)
from mlxtend.preprocessing import TransactionEncoder

In [20]:
# 初始化TransactionEncoder并将数据转换为布尔值
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
first30 = df_table["items"].head(30).values 
dataset = dataset.loc[:,first30] 
print(dataset.shape)

# 运行FP-growth算法
from mlxtend.frequent_patterns import fpgrowth
res = fpgrowth(dataset, min_support=0.05, use_colnames=True)
res.head(10)

(7500, 30)


,support,itemsets
0,0.179733,(eggs)
1,0.087200,(burgers)
2,0.062533,(turkey)
3,0.238267,(mineral water)
4,0.132000,(green tea)
5,0.129600,(milk)
6,0.058533,(whole wheat rice)
7,0.076400,(low fat yogurt)
8,0.170933,(french fries)
9,0.050533,(soup)


In [23]:
from mlxtend.frequent_patterns import association_rules
res = association_rules(res, num_itemsets=None, metric="lift", min_threshold=1)
# 根据置信度对值进行排序
res.sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3,(spaghetti),(mineral water),0.174133,0.238267,0.059733,0.343032,1.439698,1.0,0.018243,1.159468,0.369806,0.169376,0.137536,0.296866
5,(chocolate),(mineral water),0.163867,0.238267,0.052667,0.321400,1.348907,1.0,0.013623,1.122506,0.309351,0.150706,0.109136,0.271220
0,(eggs),(mineral water),0.179733,0.238267,0.050933,0.283383,1.189351,1.0,0.008109,1.062957,0.194090,0.138758,0.059228,0.248574
2,(mineral water),(spaghetti),0.238267,0.174133,0.059733,0.250699,1.439698,1.0,0.018243,1.102184,0.400941,0.169376,0.092710,0.296866
4,(mineral water),(chocolate),0.238267,0.163867,0.052667,0.221041,1.348907,1.0,0.013623,1.073398,0.339566,0.150706,0.068379,0.271220
1,(mineral water),(eggs),0.238267,0.179733,0.050933,0.213766,1.189351,1.0,0.008109,1.043286,0.209004,0.138758,0.041490,0.248574
